# Create Elevation Index

Copernicus 30m DEM on [AWS open data](https://copernicus-dem-30m.s3.amazonaws.com/readme.html) does not have a spatial index of the available tiles. We need to generate one, joining it to the spatial index provided. The polygon index can then be used to query for scenes matching an analysis aoi.


1. Download the index shapefile provided by copernicus
2. Table join index vectors with list of available files on s3
3. Save joined index
4. Create visual to determine data availability


In [18]:
import boto3
import geopandas as gpd
import os
import pandas as pd

index_dir = "/projects/maap-users/alexdevseed/"
index_zip = os.path.join(index_dir, "grid.zip")
index_shp = os.path.join(index_dir, "dem30mGrid.shp")

In [9]:
#https://copernicus-dem-30m.s3.amazonaws.com/grid.zip
# save as cop_dem_30m.zip
!wget https://copernicus-dem-30m.s3.amazonaws.com/grid.zip -O $index_zip
!unzip -d $index_dir $index_zip

--2021-03-29 22:36:39--  https://copernicus-dem-30m.s3.amazonaws.com/grid.zip
Resolving copernicus-dem-30m.s3.amazonaws.com (copernicus-dem-30m.s3.amazonaws.com)... 52.219.72.18
Connecting to copernicus-dem-30m.s3.amazonaws.com (copernicus-dem-30m.s3.amazonaws.com)|52.219.72.18|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2244913 (2.1M) [application/x-zip-compressed]
Saving to: ‘/projects/maap-users/alexdevseed/grid.zip’

/projects/maap-user 100%[===================>]   2.14M  3.38MB/s    in 0.6s    

2021-03-29 22:36:40 (3.38 MB/s) - ‘/projects/maap-users/alexdevseed/grid.zip’ saved [2244913/2244913]

Archive:  /projects/maap-users/alexdevseed/grid.zip
  inflating: /projects/maap-users/alexdevseed/dem30mGrid.dbf  
  inflating: /projects/maap-users/alexdevseed/dem30mGrid.json  
  inflating: /projects/maap-users/alexdevseed/dem30mGrid.prj  
  inflating: /projects/maap-users/alexdevseed/dem30mGrid.shp  
  inflating: /projects/maap-users/alexdevseed/dem30mGrid

In [19]:
dem_30m_index = gpd.read_file(index_shp)
dem_30m_index.head()

,id,geometry
0,Copernicus_DSM_COG_10_N85_00_W180_00_DEM,"POLYGON ((-180.001 85.000, -180.001 86.000, -1..."
1,Copernicus_DSM_COG_10_N85_00_W179_00_DEM,"POLYGON ((-179.001 85.000, -179.001 86.000, -1..."
2,Copernicus_DSM_COG_10_N85_00_W178_00_DEM,"POLYGON ((-178.001 85.000, -178.001 86.000, -1..."
3,Copernicus_DSM_COG_10_N85_00_W177_00_DEM,"POLYGON ((-177.001 85.000, -177.001 86.000, -1..."
4,Copernicus_DSM_COG_10_N85_00_W176_00_DEM,"POLYGON ((-176.001 85.000, -176.001 86.000, -1..."


In [16]:
s3 = boto3.resource('s3')
bucket = s3.Bucket('copernicus-dem-30m')
bucket_files = [s3_file.key for s3_file in bucket.objects.all()]

In [49]:

bf_df = pd.DataFrame(bucket_files, columns=["key"])
bf_df["id"] = bf_df["key"].str.split(pat="/").str[0]
bf_df["s3"] = "s3://copernicus-dem-30m/" + bf_df["key"]
bf_df.head()

,key,id,s3
0,Copernicus_DSM_COG_10_N00_00_E006_00_DEM/Coper...,Copernicus_DSM_COG_10_N00_00_E006_00_DEM,s3://copernicus-dem-30m/Copernicus_DSM_COG_10_...
1,Copernicus_DSM_COG_10_N00_00_E009_00_DEM/Coper...,Copernicus_DSM_COG_10_N00_00_E009_00_DEM,s3://copernicus-dem-30m/Copernicus_DSM_COG_10_...
2,Copernicus_DSM_COG_10_N00_00_E010_00_DEM/Coper...,Copernicus_DSM_COG_10_N00_00_E010_00_DEM,s3://copernicus-dem-30m/Copernicus_DSM_COG_10_...
3,Copernicus_DSM_COG_10_N00_00_E011_00_DEM/Coper...,Copernicus_DSM_COG_10_N00_00_E011_00_DEM,s3://copernicus-dem-30m/Copernicus_DSM_COG_10_...
4,Copernicus_DSM_COG_10_N00_00_E012_00_DEM/Coper...,Copernicus_DSM_COG_10_N00_00_E012_00_DEM,s3://copernicus-dem-30m/Copernicus_DSM_COG_10_...


In [48]:
"".join(["a",bf_df.loc[1:3, "key"].str])

TypeError: sequence item 1: expected str instance, StringMethods found